In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Modelos
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
#Ensambles
from sklearn.ensemble  import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, AdaBoostRegressor
#Búsqueda de hiper-parámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [ ]:
#Cargo los features
features = pd.read_csv("data/features.csv")
features_test = pd.read_csv("data/features_test.csv")

# (1) Modelo KNN

In [ ]:
metric = ['euclidea', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis']
weights = ['uniform', 'distance'] 
n_neighbors = [2,4,6,8,10]
knn_model = KNeighborsRegressor()